# Deferred, Delayed, and Cancelled
Section 0501 Group 04 

(**ETD: Estimated Time of Departure**): Tanmay Kotian, Pratik Kunjir, Sanika Patil, Keerthana Pramudi, Nishit Salot, Trisha Shrivastava


## Introduction

#### Project Description
Airline transportation is an integral part of travel in the 21st century, yet, despite the advances in technology that make flights easier to board, book, and track, flights still experience unforecasted interruptions in schedule as a result of different circumstances. These circumstances are usually in the form of weather delays, security delays, carrier delays, late arrival delays, etc. Flights must try to minimize delays as this aspect directly converts to flier experience which in turn affects brand image and sales. As a result, we hypothesize that flight delays will have a visible effect on carrier stocks which we hope to capture through stock data from 2018-20. In this project, we seek to draw parallels to different factors affecting delay behavior, analyze seasonal trends, and answer our questions of interest: <br/>

**1) Are there seasonal trends to flights and flight delays?** <br/>
**2) What is the relationship between delays and location?** <br/>
**3) What is the relationship between delays and carriers?** <br/>
**4) Is there a pattern to flight delays?** <br/>
**5) Do airline carriers experience a stock drop after a delay or a certain amount of delays?** <br/>

These questions are valuable in answering not only the consumer implications of booking a flight with certain airlines at certain locations, but also the resulting market implications for airline carriers as a result of delayed service.  

#### Dataset Description 
To conduct this analysis, our group utilized seven datasets. <br/>
The first dataset that we used was sourced from Kaggle. This dataset contained flight data from the United States between the years of 2018-2020, the delay circumstances listed included carrier delays, weather delays, security delays, late aircraft delays, and national air service delays. The initial dataset contained up to 28 columns and 3 million rows per year prior to filtering. As we will walk through in our processing portion, this dataset was filtered down to only contain the top five carriers in terms of flights.<br/>
The second dataset we used was sourced from datahub.io which contained airport location data. This dataset was used in order to determine where the airports were located in terms of longitude and latitude. Before processing, this dataset contained 12 columns and 57,421 rows. <br/>
Finally, the last five datasets were stock data from 2018-2020 sourced from Yahoo Finance. The data gathered from this dataset was used in this report to determine the impact of delays on carrier valuation. Each dataset contained 5 columns and 755 rows.

#### Analysis Focus
While a fair amount of processing was done in order to cleanse the data to prepare it for our analysis, our report is ultimately focused on using our data in order to provide greater insights to airline customers and companies about the impact of flight delays. Not only did we conduct exploratory analysis to determine whether were patterns, but we dug deeper and sought to understand the correlations and the impact of one event on the other. In this report, each visualization will be accompanied by additional analyses and inferences in an attempt to determine why those patterns occur.  

## Data Processing 
Data is not always structured in a way where it is easy to conduct analysis on, and, for our seven datasets, we had to different levels of processing that needed to be conducted for each dataset.

To begin our processing and analysis, we first needed to import the appropriate libraries.

In [ ]:
#import statements
import numpy as np
import pandas as pd
import datetime as dt
import networkx as nx
import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.graph_objects as go

### Airline Delay Dataset
The entirety of the airline dataset was comprised of 10 years of airline delay data for multiple flights for multiple carriers. In working with this data together, our group struggled in being able to consolidate this massive amount of data, so we decided to limit our analysis to the top 5 airline carriers in terms of flights over the last three years (2018, 2019, 2020). Each year of had up to 28 columns and 3 million rows prior to this filtering.

In [ ]:
#read in all of the data sets
read_delay_2018 = pd.read_csv(r'C:\Users\Trish\Downloads/2018.csv')
read_delay_2019 = pd.read_csv(r'C:\Users\Trish\Downloads/2019.csv')
read_delay_2020 = pd.read_csv(r'C:\Users\Trish\Downloads/2020.csv')

In [ ]:
#combine all the data sets
initial_delay_data = pd.concat([read_delay_2018,read_delay_2019,read_delay_2020])

In [ ]:
initial_delay_data.head()

In [ ]:
#determine the top 5 carriers
initial_delay_data['OP_CARRIER'].value_counts().head()

In [ ]:
#filter the 2018, 2019, 2020 data to the top 5 carriers
top_5_delay_2018 = read_delay_2018[read_delay_2018['OP_CARRIER'].isin(['WN','DL','OO','AA','UA'])]
top_5_delay_2019 = read_delay_2019[read_delay_2019['OP_UNIQUE_CARRIER'].isin(['WN','DL','OO','AA','UA'])]
top_5_delay_2020 = read_delay_2020[read_delay_2020['OP_CARRIER'].isin(['WN','DL','OO','AA','UA'])]

However, our data processing difficulties did not stop there. In the 2019 dataset, there existed only 23 columns instead of the 28 that existed in the 2018 and 2020 datasets. Furthermore, the labelling of columns in the 2019 dataset was not consistent with the 2018 and 2020 ones. To combat this, we standardized the column names and in the 2019 dataset and removed the columns missing from the 2019 dataset from the 2018 and 2020 datasets to make our final concatenated dataset. We chose to delete these as they helped reduce the size of our dataset and because we would not be able to conduct meaningful analysis with these columns over the three years. 

In [ ]:
#correct the column name from OP_UNIQUE_CARRIER to OP_CARRIER
top_5_delay_2019 = read_delay_2019.rename(columns={'OP_UNIQUE_CARRIER':'OP_CARRIER'})

In [ ]:
#save filtered data to a new csv
top_5_delay_2018.to_csv('delay_2018_top5.csv')
top_5_delay_2019.to_csv('delay_2019_top5.csv')
top_5_delay_2020.to_csv('delay_2020_top5.csv')

In [ ]:
#read in new csv files
delay_2018 = pd.read_csv('delay_2018_top5.csv')
delay_2019 = pd.read_csv('delay_2019_top5.csv')
delay_2020 = pd.read_csv('delay_2020_top5.csv')

In [ ]:
#delete the inconsistent columns from the data
del delay_2018['CRS_DEP_TIME']
del delay_2020['CRS_DEP_TIME']
del delay_2018['CRS_ARR_TIME']
del delay_2020['CRS_ARR_TIME']
del delay_2018['CRS_ELAPSED_TIME']
del delay_2020['CRS_ELAPSED_TIME']
del delay_2018['ACTUAL_ELAPSED_TIME']
del delay_2020['ACTUAL_ELAPSED_TIME']
del delay_2018['Unnamed: 0']
del delay_2019['Unnamed: 0']
del delay_2020['Unnamed: 0']
del delay_2018['Unnamed: 27']
del delay_2019['Unnamed: 20']
del delay_2020['Unnamed: 27']

In [ ]:
#concatenate the data
delay_data = pd.concat([delay_2018,delay_2019,delay_2020])

In [ ]:
#save the dataset to a new file for further work
delay_data.to_csv('delay_data.csv')

In [ ]:
delay_data.head()

In [ ]:
delay_data.reset_index()

After cleansing the dataset of the inconsistent columns, we then proceeded to focus on the data inside the flight data. Initially the arrival and departure times were stored as floats, which made them hard to understand and conduct meaningful analysis on, so we decided to change them to datetime objects. To achieve this, we first turned the floats into ints in order to be able to add the 0 in front of the times that only had 3 digits, then we converted those into strings and then to datetime objects.

In [ ]:
#convert to int
delay_data['ARR_TIME']=delay_data[['ARR_TIME']].dropna().astype(pd.Int64Dtype())
delay_data['DEP_TIME']=delay_data[['DEP_TIME']].dropna().astype(pd.Int64Dtype())

In [ ]:
#make all times 4 digits long (i.e. 00:00)
delay_data['ARR_TIME'] = delay_data['ARR_TIME'].apply('{0:0>4}'.format)
delay_data['DEP_TIME'] = delay_data['DEP_TIME'].apply('{0:0>4}'.format)

In [ ]:
#convert time into transformable objects
delay_data['ARR_TIME']=delay_data['ARR_TIME'].astype(str)
delay_data['DEP_TIME']=delay_data['DEP_TIME'].astype(str)

In [ ]:
#transform arrival and departure times to datetime with proper formatting
delay_data=delay_data.assign(ARR_TIME=pd.to_datetime(delay_data.ARR_TIME, format='%H%M', errors='coerce').dt.strftime('%H:%M'))
delay_data=delay_data.assign(DEP_TIME=pd.to_datetime(delay_data.DEP_TIME, format='%H%M', errors='coerce').dt.strftime('%H:%M'))

In [ ]:
#display transformation
delay_data[['ARR_TIME', 'DEP_TIME']].head()

Finally, after exploring our dataset, we realized that all of the delay reasons were to explain the arrival delays. To simplify our analyses for the flight delays in the future, we have created a smaller dataframe to map the arrival delays to the delay reasons.

In [ ]:
#select the late flights from delay_data
delays = delay_data[delay_data['ARR_DELAY'] > 0 ]

In [ ]:
#select the columns for analysis
delays = delays[['OP_CARRIER','FL_DATE','ARR_DELAY','CARRIER_DELAY','WEATHER_DELAY','NAS_DELAY','SECURITY_DELAY','LATE_AIRCRAFT_DELAY']]

In [ ]:
#display delay dataframe
delays.head()

### Airline Location Dataset

In order to determine the locations of flight destinations, a separate dataset sourced from datahub.io was used in order to get the appropriate longitude and latitude for each airport. Although the initial dataset was 12 columns and 57,421 rows, it was trimmed down to 3 columns and 308 rows to provide the name of the airport, its iata code, and its coordinates. The coordinates were later split into individual longitude and latitude columns to prepare for visualization.    

In [ ]:
#import the locations dataset, choosing only the relevant columns
airports = pd.read_csv('../Downloads/airport-codes_csv.csv', usecols=['name','iata_code','coordinates'])

In [ ]:
#filter the data down to the destinations in delay_data
airport_loc = airports[airports.iata_code.isin(delay_data.DEST.unique())]

In [ ]:
airport_loc.head()

In [ ]:
#create a deep copy of airport_loc to reference it in making longitude and latitude columns
airport = airport_loc.copy(deep=True)

In [ ]:
#longitude listed first in coordinates and latitude listed second
airport['long'] = airport_loc['coordinates'].str.split(',').str[0]
airport['lat'] = airport_loc['coordinates'].str.split(',').str[1]

In [ ]:
#drop extraneous coordinates column from dataset
airport.drop(columns='coordinates')

In [ ]:
#save data to a new file for further analysis
airport.to_csv('airports.csv')

### Airline Carrier Datasets 

Alongside the delay data on flights, to draw parallels on the airline stocks and delays, we have downloaded stock data from Yahoo Finance over the years of 2018, 2019 and 2020. Data for different airlines at different intervals can be found here, making it a convinient site to gather data on the filtered top five airline carriers - United Airlines, Skywest Airlines, Southwest Airlines, American Airlines and Delta Airlines. Each scraped dataset had 5 columns and around 755 rows.

Moving on to cleaning the data, we first saved the five different airline stock data into 5 .csv files. After which we created a dataframe for each file, and edited the values of the OP_CARRIER column to the respective carrier codes before concatenating all the dataframes into one. Some columns like Adj Close and Volume were irrelevent and out of scope for our praject, hence were dropped. Futhermore, we created a calculated field that better quantifies the varience in stock values called Volatility.

Once the above discussed changes and pre-processing were completed we saved the updated stock dataset as a .csv file.

In [ ]:
#Loads united airlines stock data in form of a dataframe from the UAL.csv file. 
file_path_UAL = r'C:\Users\Trish\Downloads/UAL.csv'
df_ua = pd.read_csv(file_path_UAL)

#Loads skywest airlines stock data in form of a dataframe from the SKYW.csv file. 
file_path_OO = r'C:\Users\Trish\Downloads/SKYW.csv'
df_oo = pd.read_csv(file_path_OO)

#Loads southwest airlines stock data in form of a dataframe from the LUV.csv file. 
file_path_WN = r'C:\Users\Trish\Downloads/LUV.csv'
df_wn = pd.read_csv(file_path_WN)

#Loads american airlines stock data in form of a dataframe from the AAL.csv file. 
file_path_AA = r'C:\Users\Trish\Downloads/AAL.csv'
df_aa = pd.read_csv(file_path_AA)

#Loads delta airlines stock data in form of a dataframe from the SKYW.csv file. 
file_path_DL = r'C:\Users\Trish\Downloads/DAL.csv'
df_dl = pd.read_csv(file_path_DL)

In [ ]:
#Adds a new airline carrier code column in the stock dataframe for each airline.
df_ua['OP_CARRIER'] = 'UA'
df_oo['OP_CARRIER'] = 'OO'
df_wn['OP_CARRIER'] = 'WN'
df_aa['OP_CARRIER'] = 'AA'
df_dl['OP_CARRIER'] = 'DL'

In [ ]:
#Combines all the airlines dataframe into one stock dataframe.
combined_stock_data = pd.concat([df_ua,df_oo,df_wn,df_aa,df_dl])

In [ ]:
#Drops not required columns as part of Data Cleaning.
combined_stock_data.drop(['Adj Close', 'Volume'],axis=1,inplace=True)

#Adds a stock volatility column based on high end and low end stock value.
combined_stock_data['Volatility'] = combined_stock_data['High'] - combined_stock_data['Low']

In [ ]:
combined_stock_data.head()

In [ ]:
#Creates a new csv file through the combined stock dataframe.
file_path = r'Stock_Data_Combined.csv'
combined_stock_data.to_csv(file_path)

## Data Analysis 

After processing the flight data, we were then able to begin our airline delay analysis.

### Delay Analysis By Delay Type

To begin this analysis, we will first determine the ranking of the delay types by the total amount of time that they delay flights over the course of 2018 to 2020.

In [ ]:
#sum the total delays by delay type 
CARRIER_DELAY_TOTAL = delay_data['CARRIER_DELAY'].sum()
WEATHER_DELAY_TOTAL = delay_data['WEATHER_DELAY'].sum()
NAS_DELAY_TOTAL = delay_data['NAS_DELAY'].sum()
SECURITY_DELAY_TOTAL = delay_data['SECURITY_DELAY'].sum()
LATE_AIRCRAFT_DELAY_TOTAL = delay_data['LATE_AIRCRAFT_DELAY'].sum()

In [ ]:
#create a dataframe out of the delays and their totals to conduct analysis on
DELAY_TOTAL = { ' DELAY_TYPE': ['CARRIER_DELAY','WEATHER_DELAY','NAS_DELAY','SECURITY_DELAY','LATE_AIRCRAFT_DELAY'],
                'TOTAL_DELAY_MINUTES' : [CARRIER_DELAY_TOTAL,WEATHER_DELAY_TOTAL ,NAS_DELAY_TOTAL ,SECURITY_DELAY_TOTAL,LATE_AIRCRAFT_DELAY_TOTAL]}
DELAY_COUNT = pd.DataFrame(DELAY_TOTAL)
DELAY_COUNT

In [ ]:
#finding the delay that causes the maximum delay time amongst all the delay types
print(DELAY_COUNT[DELAY_COUNT['TOTAL_DELAY_MINUTES']==DELAY_COUNT['TOTAL_DELAY_MINUTES'].max()])

##### The LATE_AIRCRAFT_DELAY causes the longest delay time of 43532158 minutes (roughly 82+ years) in total among all delays.

This aircraft delay is the consequence of the preceding flight's late arrival, which used the same plane that would be departing. This leads the present flight to depart late, setting off a chain reaction that forces passengers at other airports to board the plane considerably later as well. We can infer that late aircraft delays occur the most amongst carriers as a result of mismanagement of flight scheduling by the carrier itself, the airport the flight previously came from, or other extraneous circumstances.

### Seasonality of Flights and Flight Delays

With one of the delay causes for airlines being weather delays, we wondered if other delay types and/or flight volume during different seasons could account for the volume of delays during a particular time. In the following analysis, we seek to explore the impact of seasonality on flight delays.

To conduct this analysis, we decided to divide the seasons by the meteorological seasons as they occur in the United States, which meant that Spring is from March to May, Summer is from June to August, Fall is from September to November, and Winter is December to February.

In [ ]:
#use datetime to extract the months from the flight date
delay_data['FL_DATE'] = delay_data['FL_DATE'].astype(str).map(lambda t: dt.datetime.strptime(t, '%Y-%m-%d'))
delay_data['FL_MONTH'] = delay_data['FL_DATE'].map(lambda t: t.strftime('%B'))
delay_data['FL_YEAR'] = delay_data['FL_DATE'].map(lambda t: t.strftime('%Y'))

In [ ]:
#separate months into seasons
delay_data['SEASON'] = np.where(delay_data.FL_MONTH.isin(['March','April','May']), 'Spring',
                               np.where(delay_data.FL_MONTH.isin(['June','July','August']),'Summer',
                                       np.where(delay_data.FL_MONTH.isin(['September','October','November']),'Fall','Winter')))

In [ ]:
#flights by season in descending order
delay_data.groupby('SEASON')['FL_DATE'].count().sort_values(ascending=False)

Out of all of the seasons, Spring has the most flights. 

In [ ]:
#flights by year and season
delay_data.groupby(['FL_YEAR','SEASON'])['FL_DATE'].count().sort_values(ascending=False)

By looking at this data more closely, we can infer that the reason why Spring has the most flights in this data because it is the second most frequent season for flights and because there were not as many flights occurring in the Summer of 2020 due to the Covid-19 pandemic. 

In [ ]:
#total delays and delays by category for each season
delay_data.groupby('SEASON')[['ARR_DELAY','CARRIER_DELAY','WEATHER_DELAY','NAS_DELAY','SECURITY_DELAY','LATE_AIRCRAFT_DELAY']].sum().sort_values(by='ARR_DELAY', ascending=False)

Winter has the most delays in terms of time that the delays cost, which serves to confirm our hypothesis. However, the reason for our hypothesis does not align with the data. Winter (and all of the other season's) delays are most predominantly a result of late aircraft delays. And, even while looking at the second most costly delay by total time, carrier delay is the main culprit. From this it can be inferred that the carrier delays in Winter are a result of staff taking days off for the holidays, excessive baggage loading for those who are looking to bring gifts to family members, or extra aircraft cleaning needed during the holiday season to prevent sickness from spreading. 

### Analysis of Delays by Location

In addition to determining the frequency of delays by delay type and by season, we sought to explore the spread of delays by their location over the years 2018-2020.

#### Total Delays By Location
To find out sum of total arrival delays over 3 years for each inividual airport, the pandas groupby function was used.

In [ ]:
#find unique airports in the dataframe
delay_data['ORIGIN'].unique()    

In [ ]:
#grouping the airports present in the dataframe with their total delay over 3 years
LOCATION_DELAY = pd.DataFrame(delay_data.groupby(by = 'ORIGIN' )['ARR_DELAY'].sum())  
LOCATION_DELAY

In order to plot these airports on the map, we need the coordinates for each of these airports, so we import another csv file that contains the coordinates, iata codes for each airport, and the name of the airport.

In [ ]:
#import airline location data
location_data = pd.read_csv('../Downloads/airports.csv')
location_data.head(50)

In [ ]:
#merge the location delay dataframe withe the newly inserted dataframe on a common column IATA code
LOCATION_DELAY = LOCATION_DELAY.merge(location_data,how='left', left_on='ORIGIN', right_on='iata_code')

In [ ]:
#set the index
LOCATION_DELAY.set_index('Unnamed: 0', inplace = True)
LOCATION_DELAY

Now that we have all the required parameters for visualization, we plot the airport by their coordinates on their map, and display the sum of total delay.

In [ ]:
#use mapbox to display map
mapbox_access_token = 'pk.eyJ1IjoicHJhdGlra3VuamlyIiwiYSI6ImNrdzl5MXZmdTlqZmIybm51bzUyY3o2NWkifQ.eznHcyqjR6_PBpf_R5riog'
LOCATION_DELAY['ARR_DELAY'].astype(int)

In [ ]:
mapbox_access_token = 'pk.eyJ1IjoicHJhdGlra3VuamlyIiwiYSI6ImNrdzl5MXZmdTlqZmIybm51bzUyY3o2NWkifQ.eznHcyqjR6_PBpf_R5riog'

class_map_data = go.Scattermapbox(
        lon = LOCATION_DELAY['long'],
        lat = LOCATION_DELAY['lat'],
        text = "IATA code =" +" " + LOCATION_DELAY['iata_code'] + "," + " Total Delay =" + LOCATION_DELAY['ARR_DELAY'].astype(int).astype(str),
        hoverinfo=('text'),
        mode = 'markers',
        marker = dict(
                 
                     symbol = 'circle',
                     opacity = .6,
                     size = 7,
                     colorscale = 'Plasma',
                     cmin = 0,
                     color = LOCATION_DELAY['ARR_DELAY'],
                     cmax = LOCATION_DELAY['ARR_DELAY'].max(),
                     colorbar_title="Delays per Airport (In Minutes)"
                ) 
    
)

class_map_layout = go.Layout(
        title = 'Total Delay Time(in Minutes) for each airport',
         
        mapbox=go.layout.Mapbox(
            accesstoken=mapbox_access_token,
            zoom=1
          
        )
    )

class_map = go.Figure(data=class_map_data, layout=class_map_layout)
class_map.show()

In [ ]:
#display the locations with their corresponding arrival delay from greatest to least
LOCATION_DELAY.sort_values(by = 'ARR_DELAY', ascending= False)

From the map, airports with the most delays like "DEN", "ATL", "LAX", "DFW" have the maximum delay among all the airports (based on the top 5 carriers we choose over the years 2018-2020). We infer that this is because most of these airports host some of the busiest air corridors in the world, leading to high delay timings per aircraft. The massive amount of aircraft traffic passing in and out of these airports make them the airports with the highest delays.
Furthermore, many major airline carriers such as Hawaiian Airlines, Alaska Airlines, Delta Air Lines, United Airlines, mainly operate on these airports leading to high delay times due to the high volume of traffic.

#### Individual Delay Types By Location

While we had now determined which airports had the longest delay times, we sought to understand the impact of each individual delay type on the airports that report them to see if we could further determine the cause of flight delays.

In [ ]:
#grouping relevant data for the heatmap
heat_map_data = delay_data.groupby('ORIGIN')[['CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']].sum().sort_values(by=['CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY'], ascending=False)

In [ ]:
#Creating the heatmap
f = plt.subplots(figsize=(20, 200))
plt.tick_params(axis='both', which='major', labelsize=10, labelbottom = False, bottom=False, top = False, labeltop=True)
sns.heatmap(heat_map_data, annot=True, linewidths=.01, fmt='.2f', cbar_kws={"shrink": 0.1,}, cmap='rocket', robust=True)

At a glance, it's clear that most delays are of the type LATE AIRCRAFT DELAY or CARRIER DELAY, while delays due to security reasons have the least numbers. Carrier delays happen because there has been a mismanagement of events within the control of the carrier. These events include fueling, flight clean-up, baggage mishap amongst others.

It is the carrier's lack of insight if they were not prepared for a full house and hence failed to accomodate the time it takes for flyers to check-in, clear security checks, etc. While Airline business operations strive to be as streamlined as possible while interacting with passengers, handling passenger baggage, managing their ground staff and pilots, working in partnership with other carriers to facilitate connecting flights will always be a challenge. Companies need to cooperate and communicate to reduce information gaps and in turn bring down the number of delayed flights. Delays caused by one flight has a cascading effect on connecting flights. It is not profitable, nor does it build good customer relations, if flights were to take off on time without a bulk of their passengers who come from delayed connectings flights. 

Furthermore, Most Air Carrier delays are from airports in or around metropolitan cities. For example, ORD (O'Hare International Airport) from Chicago, ATL, DFW, and DEN are all located near metropolitan cities in the United states. Meaning, more people are travelling in the States from airports near metropolitan cities perhaps due to the concentration of businesses or, rather, people are well-off in these areas and prefer to use flights over any other means of transport.

Surprisingly, weather delays have nowhere near the numbers of the two delays discussed above. A quick google search tells how only 4% of the flights experience extreme weather delays (Bureau of Transportation Statistics). While NAS delays make up for 24% of all delays (Sheffield). Since NAS delays primarily concern operations of an airport, air traffic control, and traffic volume, most of these delays can be eliminated with better planning and perhaps with the introduction of digitization of their internal processes. With humans in-charge of operating and resolving/ caliberating flight routes and traffic, there are bound to be mishaps that lead to delays. 

Delays due to security reasons seem insignificant compared to the other delays. This could be attributed to the fact that most travellers are seasoned flyers at this age and time. And even if one isn't, they have numerous sources bombarding them with information regarding the protocols of safe flying. Furthermore, there are machines that make finding items not allowed onboard quicker. Most travellers are time-bound, goal-oriented, and hence make sure to ensure safe passage through to their destinations by following safetly rules and regulations. 

### Analysis of Delays by Carrier

#### Frequency of Carrier Delays

After exploring the delays by location, we looked towards aircraft carriers themselves to determine what delay factors impact them the most.

In [ ]:
carriers_considered=['WN','DL','OO','AA','UA']

In [ ]:
#Creating emplty lists to store maximum value of each type of delay, for each airline carrier
carrier_delay_frequency=[0,0,0,0,0]
weather_delay_frequency=[0,0,0,0,0]
nas_delay_frequency=[0,0,0,0,0]
security_delay_frequency=[0,0,0,0,0]
late_aircraft_delay_frequency=[0,0,0,0,0]
index=0
for carrier in carriers_considered:
  carrier_data=delay_data[delay_data['OP_CARRIER']==carrier]
  carrier_delay_frequency[index]=carrier_data.CARRIER_DELAY[carrier_data['CARRIER_DELAY']>0.0].count()
  weather_delay_frequency[index]=carrier_data.WEATHER_DELAY[carrier_data['WEATHER_DELAY']>0.0].count()
  nas_delay_frequency[index]=carrier_data.NAS_DELAY[carrier_data['NAS_DELAY']>0.0].count()
  security_delay_frequency[index]=carrier_data.SECURITY_DELAY[carrier_data['SECURITY_DELAY']>0.0].count()
  late_aircraft_delay_frequency[index]=carrier_data.LATE_AIRCRAFT_DELAY[carrier_data['LATE_AIRCRAFT_DELAY']>0.0].count()
  #Creating a dictionary which stores the frequency of delay types for each airline
  Delay_Frequency={'CARRIER DELAY':carrier_delay_frequency[index],
                   'WEATHER DELAY':weather_delay_frequency[index],
                   'NAS DELAY':nas_delay_frequency[index],
                   'SECURITY DELAY':security_delay_frequency[index],
                   'LATE AIRCRAFT DELAY':late_aircraft_delay_frequency[index]}
 #Displaying the type of delay which affects different carriers the most.
  print(f'The most frequent type of delay for {carrier} is {max(Delay_Frequency, key=Delay_Frequency.get)} causing {max(Delay_Frequency.values())} delays ')
  index+=1

NameError: ignored

In [ ]:
#Creating new dataframe in order to plot frequency data onto a stacked bar chart
plot_data = pd.DataFrame({'Carrier delay': carrier_delay_frequency,'Weather delay': weather_delay_frequency,
                          'NAS delay': nas_delay_frequency,'Security delay': security_delay_frequency,
                          'Late Aircraft delay': late_aircraft_delay_frequency}, index=carriers_considered)

In [ ]:
#Visualising data as a stcked bar chart
import matplotlib.pyplot as plt 
visual=plot_data.plot.barh(stacked=True,figsize=(20,10))
plt.figure(figsize=(20,8))
visual.set_ylabel('Carriers')
visual.set_xlabel('Frequency of Delays')
visual.set_title('Frequent causes of Delay for each airline')
visual.legend()


From the stacked bar chart above, we can observe that Southwest has the most delays, followed by American Airlines, SkyWest, Delta, and finally United. 

The maximum number of delays for Southwest Airlines (WN) and American Airlines (AA) are caused as a result of late arrival of the the previous flight. This delay leads to a kind of domino effect as all the other scheduled trips of that particular trip will be delayed too. This delay can be attributed to a combination of delays caused by any of the other delay types before.

For Delta (DL), American Airlines (AA), and United Airlines (UA), the most number of delays are caused due to the National Aviation System. These delays are caused due to heavy air traffic, air control delays and poor weather conditions which are not in control of the airline carriers.

From these observations, it can be seen that the greatest obstacles for carriers in terms of delays are a result of regulations or unforeseen circumstances. While the frequency of late aircraft is the greatest in terms of frequency, it is hard to necessarily determine the cause for it without tracing each aircraft back along its route. However, it can be inferred that for the carriers that have the greatest amount of delays attributed to late aircraft, their second most prevalent delay could be one of the major reasons behind the resulting delays due to their similar frequencies to the late aircraft delays. 

#### Average Carrier Delays

Despite concluding the highest frequency of delay types by carrier, we still wanted to determine which out of the 5 carriers had the most delays on average. 

Given that we already determined that arrival delay accounts for all of the individual delay types, we took the average of those delays to see which carrier had the most delays and how many minutes were elapsed on average.

In [ ]:
delay_data.groupby('OP_CARRIER')['ARR_DELAY'].mean().sort_values()

From the output above, we can see that SkyWest (OO) has the least amount of delays, followed by Southwest (WN), American Airlines (AA), Delta Airlines (DL), and finally United Airlines (UA).

From this we can determine that SkyWest Airlines is the most reliable in not having delays and United Airlines is the least reliable. In order to investigate this phenonmenon further, we sought to see the differences in strategy between the most reliable and the least reliable airline carriers based on the locations they served. 

In [ ]:
#create networks for SkyWest and United Airlines
orig_dest_OO = delay_data[delay_data['OP_CARRIER'] == 'OO'][['ORIGIN','DEST']]
orig_dest_UA = delay_data[delay_data['OP_CARRIER'] == 'UA'][['ORIGIN','DEST']]

Gl_OO = nx.from_pandas_edgelist(orig_dest_OO, "ORIGIN", "DEST")
Gl_UA = nx.from_pandas_edgelist(orig_dest_UA, "ORIGIN", "DEST")

In [ ]:
#determine the degree centrality of each location 
deg_OO = list(nx.degree(Gl_OO))
deg_UA = list(nx.degree(Gl_UA))

In [ ]:
#output degree centrality for SkyWest
sorted(deg_OO, key=lambda x: x[1], reverse=True)

In [ ]:
#output degree centrality for United
sorted(deg_UA, key=lambda x: x[1], reverse=True)

By observing the degree centrality of these two airlines' locations, it can be seen that SkyWest has a variety of origins that link to many destinations. This approach means that the locations that SkyWest serves are significantly more spread out, numerous, and varied. However, when viewing the degree centrality for United, the network that it serves is significantly more limited, clustered only on a few airports. These observations and the averages imply that having a more spread out approach when it comes to providing service not only benefits the diversity of flights and revenue opportunities available for these carriers, but also their reliability as well. This trend could be because having more destinations and thereby a larger fleet could translate to being able to cover a delayed flight, having higher priority at airports, or having mastered logistical planning as a part of strategy for greater expansion. 

### The Impact of Departure Delays on Arrival Delays

In [ ]:
#Determines the dummy variable column to assign delay staus value based on delay value.
def createDummyVariables(delay):
    if delay>0:
        return "Delayed"
    elif delay == 0:
        return "On Time"
    else:
        return "Early"
    
#Determines the dummy variable delay status columns processing.
delay_data['ARR_DELAY_STATUS'] = delay_data['ARR_DELAY'].apply(lambda delay: createDummyVariables(delay))
delay_data['DEP_DELAY_STATUS'] = delay_data['DEP_DELAY'].apply(lambda delay: createDummyVariables(delay))

#Determines the distribution of arrival delay status for each departure delay status.
pd.set_option('display.max_columns', None)
pd.crosstab(index=delay_data['ARR_DELAY_STATUS'],columns=delay_data['DEP_DELAY_STATUS'],margins=True)

The above crosstab represents the flight delay status based on their arrival and departure delay time. There are about ~34% of flights that arrive late when they are departed late. However, only ~1% of flights that depart early are delayed. Hence, it can be inferred that the wait time of the consequent flights also gets delayed when the previous flight does not depart on time, thus leading to a butterfly effect on the arrivals. Also, it can be inferred that if too many flights arrive late at the same airport it might give rise to a situation where even if the flight departed early arrive late due to the runway occupancy issues. Additionally, the previously discussed issues also might add up to increase the arrival delay time for the carriers.

### Analysis of The Relationship Between Stocks and Delays

In [ ]:
import matplotlib.pyplot as plt

#Determines the average arrival delay for each carrier based on year month.
delay_data['YEAR_MONTH'] = pd.to_datetime(delay_data['FL_DATE']).dt.to_period('M')
print('Time series Analysis:')
fig, ax = plt.subplots(figsize=(15,7))
ax = delay_data.groupby(by=['YEAR_MONTH','OP_CARRIER',])['ARR_DELAY'].mean().unstack().plot(ax=ax)
plt.xticks(rotation='vertical')
plt.xlabel('Year Month')
plt.ylabel('Arrival Delay')

The time series analysis depicts the arrival delay of the 5 airlines over a period of 3 years. Until December 2019, the average monthly delays of all airlines were very close to 0 minutes. Starting end of December 2019, we see a huge spike in delays which is consistent over 2020. We believe this is due to Covid-19 pandemic which started at the same time and impacted the airline industry heavily as borders were closing and harsher mandates were beginning to be imposed on airline carriers. After that we notice United Airlines has the highest delay in 2020 while Skywest airlines has the least delay. 

In [ ]:
fig = plt.figure(figsize=(10,10))
fig.subplots_adjust(hspace=.5)
ax = fig.add_subplot(211)
ax.plot(plot_data['FL_DATE'],plot_data['Count_Of_Delays'])
ax.xaxis.set_major_locator(dates.MonthLocator(bymonthday=1))
ax.xaxis.set_major_formatter(dates.DateFormatter('%B%Y'))
plt.xticks(rotation='vertical')
ax2 = fig.add_subplot(212)
ax2.xaxis.set_major_locator(dates.MonthLocator(bymonthday=1))
ax2.xaxis.set_major_formatter(dates.DateFormatter('%B%Y'))
ax2.plot(plot_data['FL_DATE'],plot_data['Avg'])
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
plot_data[plot_data['FL_DATE']>'1/1/2020'][['Avg','Count']].corr()

In [ ]:
plot_data[plot_data['FL_DATE']<'1/1/2020'][['Avg','Count']].corr()

By correlating the delay count with airlines stock price we see no such trend in general in earlier years. Delays were fluctuating so were the stock price. But during the time of December 2019 we see a steep rise in number of delays which continued for quite some time till May 2020. This was the era of Covid-19 and it can be the reason for increased delays. Steep Decline in stock prices are visible from February 2020 which may be the result of prolonged delay of flights. Earlier we see no such sentiment in between delay of flights and their stock price but in covid era we see that delay was actually affecting stock price of the airline. It is possible that people have been looking at delay reports about the flight and are trading based on that report. The correlation matrixes supports our inference as we see that there has been a correaltion of around 23% before 1st Jan 2020 , whereas following that we get a correlation of 67% which is significantly high.

## Conclusions

In our analysis, we explored many different potential causes for flight delays. 

While looking at seasonality overall, the Spring season had the highest frequency of flights over the last three years. We inferred that this was because Spring has temperate weather and because there is Spring break around this time. We also inferred, after looking closer at the data, that this was likely due to Covid-19 restrictions in 2020 which made Summer travel harder to do than it used to be in 2018 and 2019. Our initial hypothesis was confirmed when looking at the spread of delays by season, as Winter took the lead. However, it was not for the reason we expected, as weather made up a very small percentage of the total delays. Consistently, thoughout all the seasons, late aircraft delay accounted for the most significant percentage of delays which is most likely due to its all-encompassing nature for all late aircraft. 

Similarly, from both the location and airport end, late aircraft delay accounted for the majority of airline delays. While we know that late aircrafts are the major cause of delays within our dataset, it is hard to truly interpret the actionable solutions, as the culpability of late aircrafts can lie within other delays that preceeded the one labelled "late aircraft delay". Furthermore, aside from "late aircraft delay" and "security delay", the spread of delays amongst each carrier and location is fairly varied, so even by looking at the second most frequent delay for each category, the solutions vary. However, when analyzing each respective location and carrier alone, we can better determine the factors affecting each category and seek improvements.

From the additional analysis done on the average delay of carriers, it was determined that SkyWest had the most reliability in terms of least average delays over the years 2018-2020, followed by Southwest, American Airlines, Delta Airlines, and finally United Airlines. 

In terms of trends over time, there was a clear impact of the Covid-19 pandemic on flight delays. Outside of the number of cancelled flights increasing at this time due to health concerns, the number of delays also grew significantly. We attributed this increase to growing safety measures as a result of the pandemic taking more time to perform. 

From the previous conclusions, the evidence for the butterfly effect resulting from one flight delay to the next became more and more prevalent, leading to a more thorough investigation into the relationship between arrival and departure delays. By investigating which flights' early, on-time, and late departure times affected flights' early, on-time, and late arrival times, there was a clear correlation that accounted for 78% percent of all flights which signified that flights that left late, arrived late. The correlations between early and delayed and on-time and delayed were much weaker, with on-time still having a higher percentage of delays than early departures by 7%.

The effect of flight delays on stocks was not significant until the beginning of the knowledge of Covid-19, due the daily flight delays not being excessively long on average. However, we can conclude that in the covid era, which began on January 1, 2020, trading is being done on the sentiment of flight delays.

Through our report, we were able to analyze the trend and spread of delays through time, by carrier, and by location. In it we were able to document how Covid-19 affected many of the normal happenings of flights and even reoriented the investor sentiment to caring about flight delays per carrier. While many of us are disgruntled at the airline carriers themselves when our flights get delayed, this analysis displays that, transportation is often subject to unforeseen circumstances despite individual best practices of players within the industry to ensure the safety of its passengers. This analysis sets the groundwork for further research and analysis into the main causes of late aircraft delays for each aircraft for each carrier. The findings from this project can be further used to determine the sensitivity of other transportation services from 2018-2020 and the sensitivity of their stocks to the delays caused by the pandemic.  